## Imports

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data Sets

In [2]:
train_path = "../../ucla-stats-101-c-2024-su-regression/train.csv"

train = pd.read_csv(train_path)
train.head()

,q_demos_state,year,month,order_totals,log_total,count,count_female,count_male,count_less5,count_5to10,...,count_und25k,count_2549k,count_5074k,count_7599k,count_100149k,count_150kup,count_lessHS,count_HS,count_B,count_G
0,Alabama,2018,1,1774.01,3.248956,53,49,4,17,33,...,1,23,8,0,19,2,0,8,29,16
1,Alabama,2018,2,2015.14,3.304305,49,47,2,19,26,...,4,18,3,2,17,5,0,13,24,12
2,Alabama,2018,3,1689.01,3.227632,51,48,3,18,30,...,4,17,7,0,19,4,0,8,24,19
3,Alabama,2018,4,3303.88,3.519024,47,42,5,19,27,...,5,14,10,0,12,6,0,18,15,14
4,Alabama,2018,5,1922.96,3.283970,43,41,2,11,30,...,1,11,7,1,17,6,0,5,22,16


In [3]:
customers_path = "../../ucla-stats-101-c-2024-su-regression/customer_info_train.csv"

customers = pd.read_csv(customers_path)
customers.head()

,survey_response_id,q_demos_age,q_demos_hispanic,q_demos_race,q_demos_education,q_demos_income,q_demos_gender,q_sexual_orientation,q_demos_state,q_amazon_use_howmany,...,q_substance_use_marijuana,q_substance_use_alcohol,q_personal_diabetes,q_personal_wheelchair,q_life_changes,q_sell_your_data,q_sell_consumer_data,q_small_biz_use,q_census_use,q_research_society
0,R_2UbJL30HRjK1sdD,45 - 54 years,No,White or Caucasian,High school diploma or GED,"$100,000 - $149,999",Male,heterosexual (straight),Ohio,2,...,No,No,No,No,NaN,No,No,No,No,Yes
1,R_UPXamGKtmf4RVIZ,25 - 34 years,No,White or Caucasian,High school diploma or GED,"$25,000 - $49,999",Male,heterosexual (straight),Arkansas,1 (just me!),...,No,No,Yes,No,NaN,No,No,No,No,Yes
2,R_2dYk5auG9Fv5Qve,35 - 44 years,Yes,White or Caucasian,"Graduate or professional degree (MA, MS, MBA, ...","$50,000 - $74,999",Male,heterosexual (straight),Tennessee,1 (just me!),...,No,No,No,No,NaN,No,No,No,No,No
3,R_2aP0GyIR66gSTiR,25 - 34 years,No,White or Caucasian,High school diploma or GED,"$50,000 - $74,999",Male,heterosexual (straight),Virginia,2,...,No,Yes,No,No,NaN,No,Yes if consumers get part of the profit,I don't know,No,No
4,R_25ZREzNXsjQzRBK,35 - 44 years,No,White or Caucasian,High school diploma or GED,"$25,000 - $49,999",Male,heterosexual (straight),Utah,1 (just me!),...,No,No,Yes,No,NaN,Yes if I get part of the profit,Yes if consumers get part of the profit,I don't know,I don't know,Yes


In [4]:
amazon_path = "../../ucla-stats-101-c-2024-su-regression/amazon_order_details_train.csv"

amazon = pd.read_csv(amazon_path)
amazon.head()

,order_date,purchase_price_per_unit,quantity,shipping_address_state,title,asin_isbn_product_code,category,survey_response_id,item_cost
0,2018-12-04,7.98,1,NJ,SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...,B0143RTB1E,FLASH_MEMORY,R_01vNIayewjIIKMF,7.98
1,2018-12-22,13.99,1,NJ,Betron BS10 Earphones Wired Headphones in Ear ...,B01MA1MJ6H,HEADPHONES,R_01vNIayewjIIKMF,13.99
2,2018-12-24,8.99,1,NJ,NaN,B078JZTFN3,NaN,R_01vNIayewjIIKMF,8.99
3,2018-12-25,10.45,1,NJ,Perfecto Stainless Steel Shaving Bowl. Durable...,B06XWF9HML,DISHWARE_BOWL,R_01vNIayewjIIKMF,10.45
4,2018-12-25,10.00,1,NJ,Proraso Shaving Cream for Men,B00837ZOI0,SHAVING_AGENT,R_01vNIayewjIIKMF,10.00


## Replicate Miles Chen's Data Aggregation

Goal: recreate `train.csv` using `amazon_order_details_train.csv` and `customer_info_train.csv`

In [5]:
orders = amazon[
    ~amazon.shipping_address_state.isnull()
].groupby(
    ['order_date', 'survey_response_id'], 
    as_index=False
).agg(
    total_cost = ('item_cost', 'sum'),
    item_count = ('quantity', 'count')
).set_index(
    'survey_response_id'
).join(
    customers.set_index('survey_response_id'), 
    how='inner'
).reset_index()

orders['year'] = orders.order_date.apply(get_year)
orders['month'] = orders.order_date.apply(get_month)

orders = orders.groupby(
    ['q_demos_state', 'year', 'month'], 
    as_index=False
).agg(
    order_totals = ('total_cost', 'sum')
)

orders['log_total'] = orders.order_totals.apply(np.log10)
orders = orders[orders.log_total > 2]
orders.describe()

,year,month,order_totals,log_total
count,2942.000000,2942.000000,2942.000000,2942.000000
mean,2020.014276,6.511897,6992.187539,3.557543
std,1.412457,3.466804,8025.405040,0.559866
min,2018.000000,1.000000,100.170000,2.000738
25%,2019.000000,3.000000,1690.020000,3.227892
50%,2020.000000,7.000000,4155.750000,3.618649
75%,2021.000000,10.000000,8943.030000,3.951485
max,2022.000000,12.000000,53431.740000,4.727799


In [6]:
train[['q_demos_state', 'year', 'month', 'order_totals', 'log_total']].describe()

,year,month,order_totals,log_total
count,2942.000000,2942.000000,2942.000000,2942.000000
mean,2020.014276,6.511897,6992.187539,3.557543
std,1.412457,3.466804,8025.405040,0.559866
min,2018.000000,1.000000,100.170000,2.000738
25%,2019.000000,3.000000,1690.020000,3.227892
50%,2020.000000,7.000000,4155.750000,3.618649
75%,2021.000000,10.000000,8943.030000,3.951485
max,2022.000000,12.000000,53431.740000,4.727799


In [7]:
mc_aggregate(amazon, customers).describe()

,year,month,order_totals,log_total
count,3037.000000,3037.000000,3037.000000,3037.000000
mean,2020.003293,6.498189,6775.141785,3.497852
std,1.411879,3.454830,7990.690821,0.645738
min,2018.000000,1.000000,3.680000,0.565848
25%,2019.000000,3.000000,1516.610000,3.180874
50%,2020.000000,6.000000,3926.510000,3.594007
75%,2021.000000,9.000000,8756.710000,3.942341
max,2022.000000,12.000000,53431.740000,4.727799


### Test the pipeline using test data

In [8]:
amazon_test_path = "../../ucla-stats-101-c-2024-su-regression/amazon_order_details_test.csv"
customers_test_path = "../../ucla-stats-101-c-2024-su-regression/customer_info_test.csv"
test_path = "../../ucla-stats-101-c-2024-su-regression/test.csv"

_amazon_test = pd.read_csv(amazon_test_path)
_customers_test = pd.read_csv(customers_test_path)
_test = pd.read_csv(test_path)

_amazon_test['item_cost'] = _amazon_test['quantity']

In [9]:
_amazon_test

,order_date,quantity,shipping_address_state,category,survey_response_id,item_cost
0,2018-01-13,1,VA,BEAUTY,R_03aEbghUILs9NxD,1
1,2018-02-15,1,VA,CHOCOLATE_CANDY,R_03aEbghUILs9NxD,1
2,2018-02-16,1,VA,MEASURING_CUP,R_03aEbghUILs9NxD,1
3,2018-05-05,1,VA,NaN,R_03aEbghUILs9NxD,1
4,2018-05-26,1,VA,ABIS_BOOK,R_03aEbghUILs9NxD,1
...,...,...,...,...,...,...
896509,2021-04-01,1,MA,TOOTH_CLEANING_AGENT,R_zfqnsBzlOAKibzb,1
896510,2021-04-14,1,MA,TOWEL,R_zfqnsBzlOAKibzb,1
896511,2021-05-22,4,MA,TOOTH_CLEANING_AGENT,R_zfqnsBzlOAKibzb,4
896512,2021-12-01,4,MA,TOOTH_CLEANING_AGENT,R_zfqnsBzlOAKibzb,4


In [10]:
_orders_test = _amazon_test[
    ~_amazon_test.shipping_address_state.isnull()
].groupby(
    ['order_date', 'survey_response_id'], 
    as_index=False
).agg(
    total_cost = ('item_cost', 'sum')
).set_index(
    'survey_response_id'
).join(
    customers.set_index('survey_response_id'), 
    how='inner'
).reset_index()

_orders_test['year'] = _orders_test.order_date.apply(get_year)
_orders_test['month'] = _orders_test.order_date.apply(get_month)

_orders_test = _orders_test.groupby(
    ['q_demos_state', 'year', 'month'], 
    as_index=False
).agg(
    order_totals = ('total_cost', 'sum')
)

_orders_test['log_total'] = _orders_test.order_totals.apply(np.log10)
_orders_test.describe()

,order_totals,log_total
count,0.0,0.0
mean,NaN,NaN
std,NaN,NaN
min,NaN,NaN
25%,NaN,NaN
50%,NaN,NaN
75%,NaN,NaN
max,NaN,NaN


In [20]:
selected_set = set()

for id, (state, year, month) in _test[['q_demos_state', 'year', 'month']].iterrows():
    row = (state, year, month)
    selected_set.add(row)

len(selected_set)

2952

In [28]:
_agg = mc_aggregate(_amazon_test, _customers_test)[['q_demos_state', 'year', 'month']]

for id, (state, year, month) in _agg.iterrows():
    row = (state, year, month)
    if row not in selected_set:
        _agg = _agg.drop(id)

agg_set = set()

for id, (state, year, month) in _agg.iterrows():
    row = (state, year, month)
    agg_set.add(row)

agg_set == selected_set

True

In [36]:
_agg.reset_index(drop=True)

,q_demos_state,year,month
0,Alabama,2018,1
1,Alabama,2018,2
2,Alabama,2018,3
3,Alabama,2018,4
4,Alabama,2018,5
...,...,...,...
2947,Wyoming,2021,4
2948,Wyoming,2021,6
2949,Wyoming,2021,7
2950,Wyoming,2021,12


In [39]:
drop_unwanted_rows(mc_aggregate(_amazon_test, _customers_test)[['q_demos_state', 'year', 'month', 'log_total']], _test)

,q_demos_state,year,month,log_total
0,Alabama,2018,1,1.838849
1,Alabama,2018,2,1.845098
2,Alabama,2018,3,1.903090
3,Alabama,2018,4,1.924279
4,Alabama,2018,5,1.949390
...,...,...,...,...
2947,Wyoming,2021,4,1.204120
2948,Wyoming,2021,6,0.778151
2949,Wyoming,2021,7,1.462398
2950,Wyoming,2021,12,1.079181
